# Новосибирск

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline

In [43]:
def to_string(a):
    if isinstance(a, float):
        return str(int(a))
    elif isinstance(a, int):
        return str(a)
    else:
        return str(a)

DF = pd.DataFrame()    
   
for file in os.listdir('.'):
    if 'xlsx' in file and '$' not in file:
        name = file[:file.find('.xlsx')]
        temp_df = pd.read_excel(file, usecols=['Дата и время', 'Номер абонента'], skiprows=4, dtype=str)
        temp_df['Кампания'] = name
        DF = DF.append(temp_df)


            
         
DF = DF[~DF['Номер абонента'].isnull()]  
DF['Дата и время'] = pd.to_datetime(DF['Дата и время'], format='%Y-%m-%d %H:%M:%S')





In [44]:
from pprint import pprint
s = '''+79830028188 (Терапевт нск);
+79830028187 (Терапевт нск);
+79231150775 (Терапевт нск);
+79059590638 (НСК_Удаление родинок_Новосибирск);
+79137099367 (Педиатр нсб);
+79133728033 (НСК_Удаление родинок_Новосибирск);
+77071906966 (НСК_Удаление родинок_Новосибирск);
+79231116229 (НСК_Удаление родинок_Новосибирск);
+79529049670 (НСК_Удаление родинок_Новосибирск);
+79618753867 (НСК_Удаление родинок_Новосибирск);
+79628409228 (НСК_Удаление родинок_Новосибирск);
+79163950593 (Врач на дом нсб #2);
+79232212893 (НСК_Удаление родинок_Новосибирск);
+79231285636 (Педиатр нсб);
+79039977668 (Терапевт нск);
+79833073205 (Врач на дом нсб #2);
+79125929806 (Педиатр нсб);
+79131245177 (НСК_Удаление родинок_Новосибирск);
+79137232242 (Врач на дом нсб #2);
+79537657748 (ЛОР нск);'''

s = [i.strip(';').replace('(', '').replace(')', '').split(maxsplit=1) for i in s.split('\n')]

from difflib import get_close_matches

lst = DF['Кампания'].to_list()

l = [[i[0], get_close_matches(i[1], lst, n=1, cutoff=0.8)[0]] for i in s]

x = pd.DataFrame([pd.Series([j[0] for j in l], name='Номер абонента'), pd.Series([j[1] for j in l], name='Кампания')]).T

x['Дата и время'] = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

DF = DF.append(x) 




In [45]:
DF = DF.sort_values(by='Дата и время', ignore_index=True)

DF = DF.drop_duplicates(subset='Номер абонента', ignore_index=True) 



In [46]:
def join_list(lst):
    return ', '.join([str(i) for i in lst])



au_telefons = pd.read_excel('./Оборот/НСК_оборот.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)                 

lst_with_coincidenc_telefon  = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append('')
    for i in DF['Номер абонента']:
        if str(i) in row:
            lst_with_coincidenc_telefon[index] = str(i)
            break
            
au_telefons['Совпадение'] = lst_with_coincidenc_telefon            
            
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']
au_telefons_final = au_telefons_final.merge(DF, left_on='Совпадение', right_on='Номер абонента', how='left')
au_telefons_final = au_telefons_final.fillna('')


# не забыть проверить чтобы даты приемов были больше или равны дате звонка          



In [47]:
au_telefons_final['Совпадение дат'] = au_telefons_final['Дата приема'] >= au_telefons_final['Дата и время']

au_telefons_final = au_telefons_final[au_telefons_final['Совпадение дат'] == True]

au_telefons_final.to_excel('НСК_обработано.xlsx')